# **Chapter 5: Formatting Output & Speaking for Claude**

---
**Lesson:**

Claude can format its output in a wide variety of ways. You just need to ask for it to do so!
One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to Claude. It turns out, you can also use XML tags to make Claude's output clearer and more easily understandable to humans.

---

First we will setup our dependencies

In [3]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#Set up model inference modifiers, and ensure Claude v2 is being called from Bedrock 
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

# **Examples:**

**Example 5.1 - Tagging**

Remember the 'poem preamble problem' we solved in Chapter 2 by asking Claude to skip the preamble entirely? It turns out we can also achieve a similar outcome by telling Claude to put the poem in XML tags.

In [4]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Rabbit")

response = textgen_llm(prompt)

print_ww(response)

 Here is a haiku about Rabbit in <haiku> tags:

<haiku>
Floppy ears twitching
Nose wiggling, whiskers flutter
Rabbit hops away
</haiku>


**Example 5.2 - Half Tagging**

Why is this something we'd want to do? Well, having the output in XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags.		

An extension of this technique is to put the first XML tag AFTER "Assistant:". When you put text after "Assistant:", you're basically telling Claude that Claude has already said something, and that it should continue from that point onward.
Below, we've done this with the first "haiku" XML tag.		

In [5]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags

Assistant: <haiku>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Cat")

response = textgen_llm(prompt)

print_ww(response)


Cat purrs softly, curled
Up napping on sunny couch
Peaceful creature
</haiku>


**Example 5.3 - JSON**

Claude also excels at using other output formatting styles, notably JSON.

In [6]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Use JSON format with the keys as "first_line", "second_line", and "third_line"

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Cat")

response = textgen_llm(prompt)

print_ww(response)

 Here is a haiku about Cat in JSON format:

```json
{
  "first_line": "Fur so soft and warm",
  "second_line": "Curious, playful kitten",
  "third_line": "Likes to chase string toys"
}
```


**Example 5.4 - Combination**

Below is an example of multiple input variables in the same prompt AND output formatting specification, all done using XML tags.		

In [7]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL","ADJECTIVE"], 
    template="""

Human: Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE} email> XML tags.

Assistant: <{ADJECTIVE}_email>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.", ADJECTIVE="Olde English")

response = textgen_llm(prompt)

print_ww(response)


Greetings good Sir Zack,
Prithee I sendeth this missive to gently inquire regarding yon prompt which thou wert charged to
scribe. If it wouldst please thee, prithee sendeth me word at thy earliest convenience. I await thy
reply with bated breath.
Godspeed,
Claude
</Olde_English_email>


# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 5.1 - Steph Curry GOAT**

Forced to make a choice, Claude sometimes designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?
Modify the "best basketball player" prompt below and use the "speaking for Claude" technique to make Claude make a detailed argument that the best basketball player of all time is Stephen Curry.

In [11]:
response = textgen_llm("""

Human: Who is the overall best basketball player of all time? Make the case for one player.

Assistant:""")

print_ww(response)

 There is no definitive consensus on the single best basketball player of all time, as there are
reasonable cases that can be made for several legendary players. However, here is an argument for
Michael Jordan being the best ever:

- Unmatched resume of achievements: 6 NBA championships, 6 NBA Finals MVPs, 5 regular season MVPs,
10-time scoring champion, 9-time All-Defensive First Team selection, and many other accolades.
Arguably the most decorated player ever.

- Dominant peak: Jordan had an incredible peak from around 1987-1993 where he was putting up insane
stat lines of over 30 points, 6 rebounds and 5 assists per game while also playing elite defense. He
looked superhuman at times with his athleticism and skills.

- Clutch performer: Jordan made numerous iconic buzzer beaters and thrived under pressure. He was at
his best when the game was on the line, epitomized by the "Flu Game" and his Finals performances.

- Transcendent star power and global impact: Jordan became an internat

**Exercise 5.2 - Two Haikus**

Modify the haiku prompt in the highlighted template box below and use XML tags so that Claude writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [9]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags

Assistant: <haiku>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Cat")

response = textgen_llm(prompt)

print_ww(response)


Cat purrs softly, curled
Up napping on the windowsill
Sunbeams make her glow
</haiku>


**Exercise 5.3 - Two Haikus, Two Animals**

Modify the haiku prompt in the cell below so that Claude produces two haikus about two different animals. 
Use {ANIMAL1} as a stand-in for the first substitution, and {ANIMAL2} as a stand-in for the second substitution.

In [10]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], #Change Input variables 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags 

Assistant: <haiku>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL1="Cat",ANIMAL2="Dog") #We want these to be inputs to the prompt

response = textgen_llm(prompt)

print_ww(response)

KeyError: 'ANIMAL'

# Chapter 5 - END.